# 동작구 정책 제안

In [1]:
#패키지 import
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from time import sleep
import requests
import re
import pandas as pd
import numpy as np

## 1. 대한민국 구석구석에서 한국 역사 여행지 목록 불러오기

한국관광공사에서 여행정보를 제공하는 페이지인 '대한민국 구석구석'에서 전국의 여행지 목록을 열람할 수 있는데, 그 중 "역사"태그로 역사관련 문화유적지의 목록을 찾을 수 있다. 아래는 역사 관광지의 목록을 크롤링하는 코드이다.

### Selenium과 BeautifulSoup로 parsing하기

In [ ]:
header = {'User-Agent': ''}
d = webdriver.Chrome(executable_path="C:/Users/jmlee/Code/chromedriver_win32/chromedriver.exe") #웹앱을 테스트하는데 이용하는 프레임워크, webdriver라는 API를 통해 운영체제에 설치된 브라우저 제어 
d.implicitly_wait(3)

#페이지 불러오기
d.get('https://korean.visitkorea.or.kr/main/main.do') #한국관광공사 대한민국구석구석 메인사이트
d.find_element_by_xpath('/html/body/div[7]/div/button').click() # 코로나 바이러스 팝업 '닫기'클릭
d.find_element_by_xpath('/html/body/div[5]/ul/li[2]/a/em').click() # All여행지 항목 클릭
d.find_element_by_xpath('/html/body/div[3]/div[1]/div[2]/div/ul/li[10]/a').click() # 역사 태그 클릭

req = requests.get('https://korean.visitkorea.or.kr/list/ms_list.do?choiceTag=%EC%97%AD%EC%82%AC&choiceTagId=d3fd4d9f-fbd4-430f-b5d5-291b4d9920be')
html = req.text #html소스 가져오기, python의 문자열 객체를 반환
soup = BeautifulSoup(html, "html.parser") #html코드를 python이 이해하는 객체 구조로 변환하는 parsing으로 의미있는 정보 추출 가능

### 관광지 목록 불러오기

In [2]:
click_idx=0
f=open('placeList.txt','w',-1,'UTF-8') #역사 관광지의 이름 목록을 크롤링해 txt문서와 place 리스트에 저장한다
#f1=open('imgList.txt','w',-1,'UTF-8')

place = [] #장소명 저장하는 리스트
#img = [] #이미지 링크 저장하는 리스트

for j in range(1, 385): #384페이지까지 있음
    
    
    for i in range(1,11): #한페이지 당 열개의 관광지 목록 
    
            place_xpath = '//*[@id="contents"]/div[2]/div[1]/ul/li['+str(i)+']/div[2]/div/a' #관광지 이름 path
            #img_xpath = '//*[@id="contents"]/div[2]/div[1]/ul/li['+str(i)+']/div[1]/a/img' #관광지 사진 링크 path
            p = ""
            #img = ""
            try:
                p = d.find_element_by_xpath(place_xpath)
                #img = d.find_element_by_xpath(img_xpath)
                place.append(p.text)
                
                f.write(p.text+'\n')
                #f1.write(img.text+'\n')
                img.append(img)

            except:
                pass 
            
    #다음버튼 클릭하기(1,2,3,4,5 클릭하고 다음버튼), 다음페이지로 넘어가기

    if j<=5: #1~5페이지까지는 <<,<버튼이 존재하지 않아서 클릭할 인덱스가 다르다
        page_box=d.find_element_by_class_name('page_box')
        page_num=page_box.find_elements_by_tag_name('a')
        page_num[j].click()
        sleep(5)
    else:
        page_box=d.find_element_by_class_name('page_box')
        page_num=page_box.find_elements_by_tag_name('a')
        page_num[3+click_idx].click()
        click_idx+=1
        sleep(5)
        if click_idx==5: click_idx=0
    

f.close()        

print (place)
#print (img)

NameError: name 'd' is not defined

배열 place에 '대한민국 구석구석' 의 관광지 목록 중 #역사 인 관광지들의 이름을 모두 저장하였다.

In [ ]:
print(place) 

## 2. 불러온 관광지 목록을 네이버지도에 검색하여 주변 관광요소 크롤링하기

네이버 지도에서 관광지를 검색하면 지도에서 그 관광지의 위치가 검색되고, 관광지 주변의 음식점, 카페, 숙박, 디저트, 엔터테인먼트, 대중교통에 대한 정보를 각각 크롤링할 수 있음

In [2]:
#리스트 불러오기 필요

place=['읍내동 마애불상','경복궁'] #예시
search_kor=['공원','정원','주차시설','술집','화장실','관광명소','쇼핑']

In [3]:
#데이터 프레임생성, 각 열의 이름 선언 (장소명,관광요소 종류, 관광요소 이름)
df=pd.DataFrame(columns=['place_name','tour_element','tour_element_name'])

### html path 저장
xpath가 바꼈을 때를 대비하여 코드에서 사용되는 html xpath들을 저장

In [46]:
# 팝업창 닫기
popup_close='//*[@id="intro_popup_close"]/span'

#네이버 지도 홈으로 돌아가기
NmapHome='/html/body/app/layout/div[2]/div[1]/navbar/perfect-scrollbar/div/div[1]/div/ul/li[1]/a/span'

#네이버 지도 홈 검색버튼
NmapHome_search='//*[@id="sidebar"]/navbar/perfect-scrollbar/div/div[1]/div/ul/li[2]/a/span[1]'

#네이버 지도 홈 검색창
NmapHome_searchbox='/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-box/div/div/div/input'

#검색 결과 없음
search_error='/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/h2'

#음식점
restaurants_path='/html/body/app/layout/div[2]/div[2]/div[1]/app-base/search-box/div/div[3]/div/around-here-card/ul/li[1]/a/span[1]'

#카페
cafes_path='/html/body/app/layout/div[2]/div[2]/div[1]/app-base/search-box/div/div[3]/div/around-here-card/ul/li[2]/a/span[1]'

#전체보기 
fullElem_path='/html/body/app/layout/div[2]/div[2]/div[1]/app-base/search-box/div/div[3]/div/around-here-card/ul/li[13]/button/span'

#숙박
lodges_path='/html/body/app/layout/div[2]/div[2]/div[1]/app-base/search-box/div/div[3]/div/around-here-card/ul/li[9]/a/span[1]'

#디저트
desserts_path='/html/body/app/layout/div[2]/div[2]/div[1]/app-base/search-box/div/div[3]/div/around-here-card/ul/li[7]/a/span[1]'

#엔터테인먼트
enter_path='/html/body/app/layout/div[2]/div[2]/div[1]/app-base/search-box/div/div[3]/div/around-here-card/ul/li[10]/a/span[1]'

#대중교통
trans_path='/html/body/app/layout/div[2]/div[2]/div[1]/app-base/search-box/div/div[3]/div/around-here-card/ul/li[4]/a/span[1]'

#종류
tag='/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[2]/span'


### 함수 crawlNaverMap
> 음식점, 카페, 숙박, 디저트, 엔터, 대중교통 순으로 크롤링하는 함수

In [5]:
def crawlNaverMap():
    try:
            
        #음식점
        d.find_element_by_xpath(restaurants_path).click() #음식점 섹션 클릭
        sleep(2)
        i=0
            
        while True:
            try:
                r.append(d.find_elements_by_class_name('search_title_text')[i].text) #음식점 이름이 담겨있는 class를 가져와서 하나씩 리스트에 append

                if (i+1)%3==0: #화면에 3개씩 보이도록 스크롤
                    ELEMENT = d.find_elements_by_class_name('search_title_text')[i+1] 
                    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                i+=1
                        
            except: break
            
        d.find_element_by_xpath(NmapHome).click() #지도 홈 클릭
    

        #카페
            
        d.find_element_by_xpath(cafes_path).click() #카페 섹션 클릭
        sleep(2) 
        i=0
            
        while True:
            try:
                c.append(d.find_elements_by_class_name('search_title_text')[i].text)
        
                if (i+1)%3==0: #화면에 3개씩 보이도록 스크롤
                    ELEMENT = d.find_elements_by_class_name('search_title_text')[i+1] 
                    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)                    
                i+=1
                        
            except: break
           
        d.find_element_by_xpath(NmapHome).click() #지도 홈 클릭     
        d.find_element_by_xpath(fullElem_path).click() #전체보기     
                
        #숙박
            
        d.find_element_by_xpath(lodges_path).click()
        sleep(2)
        i=0
        while True:
            try:
                l.append(d.find_elements_by_class_name('search_title_text')[i].text)
                             
        
                if (i+1)%3==0: #화면에 3개씩 보이도록 스크롤
                    ELEMENT = d.find_elements_by_class_name('search_title_text')[i+1] 
                    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        
                i+=1
            except: break
             
        d.find_element_by_xpath(NmapHome).click() #지도 홈 클릭
        d.find_element_by_xpath(fullElem_path).click() #전체보기     
            
        #디저트        
                    
        d.find_element_by_xpath(desserts_path).click()
        sleep(2)
        i=0
        while True:
            try:
                dss.append(d.find_elements_by_class_name('search_title_text')[i].text)
        
                if (i+1)%3==0: #화면에 3개씩 보이도록 스크롤
                    ELEMENT = d.find_elements_by_class_name('search_title_text')[i+1] 
                    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        
                i+=1
                        
            except: break
                
        d.find_element_by_xpath(NmapHome).click() #지도 홈 클릭
        d.find_element_by_xpath(fullElem_path).click() #전체보기     
           
        #엔터테인먼트
            
        d.find_element_by_xpath(enter_path).click() 
        
        #영화관, 코인노래방, 노래방, 볼링장, 당구장, 실내골프연습장 선택
        d. find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/around-here-layout/around-here-home/div/div[1]/ul/li[2]/button').click()
        d. find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/around-here-layout/around-here-home/div/div[1]/ul/li[3]/button').click()
        d. find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/around-here-layout/around-here-home/div/div[1]/ul/li[4]/button').click()
        d. find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/around-here-layout/around-here-home/div/div[1]/ul/li[5]/button').click()
        d. find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/around-here-layout/around-here-home/div/div[1]/ul/li[6]/button').click()
        d. find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/around-here-layout/around-here-home/div/div[1]/ul/li[7]/button').click()
        
        sleep(2)
        i=0
        while True:
            try:
                e.append(d.find_elements_by_class_name('search_title_text')[i].text)
        
                if (i+1)%3==0: #화면에 3개씩 보이도록 스크롤
                    ELEMENT = d.find_elements_by_class_name('search_title_text')[i+1] 
                    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        
                i+=1
                        
            except: break
        
        d.find_element_by_xpath(NmapHome).click() #지도 홈 클릭
        
        #대중교통
        d.find_element_by_xpath(trans_path).click()

        #버스정류장
        sleep(2)
        i=0
        while True:
            try:
                t.append(d.find_elements_by_class_name('search_title_text')[i].text)
        
                if (i+1)%3==0: #화면에 3개씩 보이도록 스크롤
                    ELEMENT = d.find_elements_by_class_name('search_title_text')[i+1] 
                    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        
                i+=1
                        
            except: break
                
        #지하철역
        d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/around-here-layout/around-here-home/div/div[1]/ul/li[2]/button').click()
        sleep(2)
        i=0
        while True:
            try:
                t.append(d.find_elements_by_class_name('search_title_text')[i].text)
        
                if (i+1)%3==0: #화면에 3개씩 보이도록 스크롤
                    ELEMENT = d.find_elements_by_class_name('search_title_text')[i+1] 
                    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        
                i+=1
                        
            except: break
        
        #공공자전거
        d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/around-here-layout/around-here-home/div/div[1]/ul/li[3]/button').click()
        sleep(2)
        i=0
        while True:
            try:
                t.append(d.find_elements_by_class_name('search_title_text')[i].text)
        
                if (i+1)%3==0: #화면에 3개씩 보이도록 스크롤
                    ELEMENT = d.find_elements_by_class_name('search_title_text')[i+1] 
                    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                        
                i+=1
                        
            except: break
                
    except:pass


In [133]:
def crawlNaverMap2():
    #공원
    try:
        elem=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-box/div/div/div/input')
        elem.clear()
        elem.send_keys(idx+' 주변 '+ search_kor[0])
        d.find_element_by_xpath(NmapHome_searchbox).send_keys(Keys.ENTER) #엔터 치기
        sleep(2)

        print(idx+' 주변 '+search_kor[0]+'를 가져옵니다.') 
        while True: #페이지 끝날때 까지
            try:
                for j in range(1,21):
                    tag=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[2]/span').text
                    if(tag=='근린공원'):
                        park=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[1]/strong/span').text
                        p.append(park)
                        print(park)
                        if j%3==0: #화면에 3개씩 보이도록 스크롤
                            ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                            d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                    else:
                        if j%3==0: #화면에 3개씩 보이도록 스크롤
                            ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                            d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                next_button=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]')
                next_button.click()
                sleep(3)
            except: break
    except: pass
                
    #정원
    try:
        elem=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-box/div/div/div/input')
        elem.clear()
        elem.send_keys(idx+' 주변 '+ search_kor[1])
        d.find_element_by_xpath(NmapHome_searchbox).send_keys(Keys.ENTER) #엔터 치기
        sleep(2)
        print(idx+' 주변 '+search_kor[1]+'를 가져옵니다.') 
        while True: #페이지 끝날때 까지
            try:
                for j in range(1,21):
                    tag=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[2]/span').text
                    if(tag=='정원'):
                        garden=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[1]/strong/span').text
                        g.append(garden)
                        print(garden)
                        if j%3==0: #화면에 3개씩 보이도록 스크롤
                            ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                            d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                    else:
                        if j%3==0: #화면에 3개씩 보이도록 스크롤
                            ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                            d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                next_button=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]')
                next_button.click()
                sleep(3)
            except: break
    except: pass
                
#     #주차시설
#     try:
#         elem=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-box/div/div/div/input')
#         elem.clear()
#         elem.send_keys(idx+' 주변 '+ search_kor[2])
#         d.find_element_by_xpath(NmapHome_searchbox).send_keys(Keys.ENTER) #엔터 치기
#         sleep(2)
#         print(idx+' 주변 '+search_kor[2]+'를 가져옵니다.') 
#         while True: #페이지 끝날때 까지
#             try:
#                 for j in range(1,21):
#                     tag=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[2]/span').text
#                     if(tag=='주차장'):
#                         parking=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[1]/strong/span').text
#                         pl.append(parking)
#                         print(parking)
#                         if j%3==0: #화면에 3개씩 보이도록 스크롤
#                             ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
#                             d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
#                             print(j)
#                     else:
#                         if j%3==0: #화면에 3개씩 보이도록 스크롤
#                             ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
#                             d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
#                             print(j)
#                 next_button=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]')
#                 next_button.click()
#                 sleep(3)
#             except: break
#     except: pass
                
    #술집
    try:
        elem=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-box/div/div/div/input')
        elem.clear()
        elem.send_keys(idx+' 주변 '+ search_kor[3])
        d.find_element_by_xpath(NmapHome_searchbox).send_keys(Keys.ENTER) #엔터 치기
        sleep(2)
        print(idx+' 주변 '+search_kor[3]+'를 가져옵니다.') 
        while True: #페이지 끝날때 까지
            try:
                for j in range(1,21):
                    tag=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[2]/span').text
                    if(tag=='술집'):
                        bar=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[1]/strong/span').text
                        b.append(bar)
                        print(bar)
                        if j%3==0: #화면에 3개씩 보이도록 스크롤
                            ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                            d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                    else:
                        if j%3==0: #화면에 3개씩 보이도록 스크롤
                            ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                            d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                next_button=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]')
                next_button.click()
                sleep(3)
            except: break
    except: pass
                
    #화장실
    try:
        elem=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-box/div/div/div/input')
        elem.clear()
        elem.send_keys(idx+' 주변 '+ search_kor[4])
        d.find_element_by_xpath(NmapHome_searchbox).send_keys(Keys.ENTER) #엔터 치기
        sleep(2)
        print(idx+' 주변 '+search_kor[4]+'를 가져옵니다.') 
        while True: #페이지 끝날때 까지
            try:
                for j in range(1,21):
                    tag=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[2]/span').text
                    if(tag=='화장실'):
                        toilet=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[1]/strong/span').text
                        tl.append(toilet)
                        print(toilet)
                        if j%3==0: #화면에 3개씩 보이도록 스크롤
                            ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                            d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                    else:
                        if j%3==0: #화면에 3개씩 보이도록 스크롤
                            ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                            d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                next_button=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]')
                next_button.click()
                sleep(3)
            except: break
    except: pass
        
    #관광명소
    try:
        elem=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-box/div/div/div/input')
        elem.clear()
        elem.send_keys(idx+' 주변 '+ search_kor[5])
        d.find_element_by_xpath(NmapHome_searchbox).send_keys(Keys.ENTER) #엔터 치기
        sleep(2)
        print(idx+' 주변 '+search_kor[5]+'를 가져옵니다.') 
        while True: #페이지 끝날때 까지
            try:
                for j in range(1,21):
                    tag=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[2]/span').text
                    if(tag=='관광명소'):
                        trip=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[1]/strong/span').text
                        tr.append(trip)
                        print(trip)
                        if j%3==0: #화면에 3개씩 보이도록 스크롤
                            ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                            d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                    else:
                        if j%3==0: #화면에 3개씩 보이도록 스크롤
                            ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                            d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                next_button=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]')
                next_button.click()
                sleep(3)
            except: break
    except: pass
                
    #쇼핑
    try:
        elem=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-box/div/div/div/input')
        elem.clear()
        elem.send_keys(idx+' 주변 '+ search_kor[6])
        d.find_element_by_xpath(NmapHome_searchbox).send_keys(Keys.ENTER) #엔터 치기
        sleep(2)
        print(idx+' 주변 '+search_kor[6]+'를 가져옵니다.') 
        while True: #페이지 끝날때 까지
            try:
                for j in range(1,21):
                    tag=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[2]/span').text
                    if(tag=='쇼핑'):
                        shopping=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[1]/strong/span').text
                        s.append(shopping)
                        print(shopping)
                        if j%3==0: #화면에 3개씩 보이도록 스크롤
                            ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                            d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                    else:
                        if j%3==0: #화면에 3개씩 보이도록 스크롤
                            ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                            d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
                next_button=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]')
                next_button.click()
                sleep(3)
            except: break
    except: pass

In [127]:
def add_dataframe(n,idx,r,c,dss,l,t,e,p,g,pl,b,tl,tr,s):
    #n=한 장소당 데이터의 개수
    for i in range(len(r)):
        df.loc[n]=[place,'음식점',r[i]]
        n+=1
        i+=1
   
    for i in range(len(c)):
        df.loc[n]=[place,'카페',c[i]]
        n+=1
        i+=1
        
    for i in range(len(dss)):
        df.loc[n]=[place,'디저트',dss[i]]
        n+=1
        i+=1
        
    for i in range(len(l)):
        df.loc[n]=[place,'숙소',l[i]]
        n+=1
        i+=1
        
    for i in range(len(t)):
        df.loc[n]=[place,'대중교통',t[i]]
        n+=1
        i+=1
        
    for i in range(len(e)):
        df.loc[n]=[place,'엔터테인먼트',e[i]]
        n+=1
        i+=1
        
    for i in range(len(p)):
        df.loc[n]=[place,'공원',p[i]]
        n+=1
        i+=1
        
    for i in range(len(g)):
        df.loc[n]=[place,'정원',g[i]]
        n+=1
        i+=1
        
    for i in range(len(pl)):
        df.loc[n]=[place,'주차시설',pl[i]]
        n+=1
        i+=1
        
    for i in range(len(b)):
        df.loc[n]=[place,'술집',b[i]]
        n+=1
        i+=1
        
    for i in range(len(tl)):
        df.loc[n]=[place,'화장실',tl[i]]
        n+=1
        i+=1
        
    for i in range(len(tr)):
        df.loc[n]=[place,'관광명소',tr[i]]
        n+=1
        i+=1
        
    for i in range(len(s)):
        df.loc[n]=[place,'쇼핑',s[i]]
        n+=1
        i+=1
        
    return n #데이터의 개수 업데이트

### place를 검색하는 코드

In [134]:
header = {'User-Agent': ''}
d = webdriver.Chrome(executable_path="C:/Users/jmlee/Code/chromedriver_win32/chromedriver.exe") #웹앱을 테스트하는데 이용하는 프레임워크, webdriver라는 API를 통해 운영체제에 설치된 브라우저 제어 
d.implicitly_wait(3)
d.get('https://map.naver.com/') 
sleep(2)
html=d.page_source
soup=BeautifulSoup(html, "html.parser")
sleep(1)

In [135]:
#페이지 불러오기
#<input _ngcontent-lqn-c13="" class="input_search ng-valid ng-dirty ng-touched" type="text" id="input_search1586413975870">
#리스트의 장소 검색

r=[] #음식점
c=[] #카페
l=[] #숙박
dss=[] #디저트
e=[] #엔터테인먼트
t=[] #교통
                
p=[] #공원
g=[]  #정원
pl=[] #주차시설
b=[] #술집 
tl=[]#화장실
tr=[]#관광명소
s=[]#쇼핑

n=0 #장소 당 데이터의 개수

d.find_element_by_xpath(popup_close).click() #팝업 닫기
d.find_element_by_xpath(NmapHome_search).click() # 검색 항목 클릭
sleep(2)
    
for idx in place:
    r=[] #음식점
    c=[] #카페
    l=[] #숙박
    dss=[] #디저트
    e=[] #엔터테인먼트
    t=[] #교통
    p=[] #공원
    g=[]  #정원
    pl=[] #주차시설
    b=[] #술집 
    tl=[]#화장실
    tr=[]#관광명소
    s=[]#쇼핑
    
    elem= d.find_element_by_xpath(NmapHome_searchbox) 
    elem.clear()
    elem.send_keys(idx) #키워드 입력(경복궁,광화문 등등)
    elem.send_keys(Keys.ENTER) #엔터 치기
    sleep(3)

    try: #에러 나는경우 > 검색 결과가 없음 >모든 리스트에 -1넣기
        print(idx+'를 확인 중입니다.')
        d.find_element_by_xpath(search_error) #에러메세지 paragraph
        print(idx+'이/가 없습니다.')
        r=[-1]
        c=[-1]
        l=[-1]
        dss=[-1]
        e=[-1]
        t=[-1]
        
        p=[-1] #공원
        g=[-1]  #정원
        pl=[-1] #주차시설
        b=[-1] #술집 
        tl=[-1]#화장실
        tr=[-1]#관광명소
        s=[-1]#쇼핑
        n=add_dataframe(n,idx,r,c,dss,l,t,e,p,g,pl,b,tl,tr,s)
        sleep(3)
        
    except: #검색 결과 있음:에러메세지 출력되지 않음(except한 경우) > parsing 후 crawl 함수 실행   
        print(idx+'이/가 존재합니다.')
#         elem.clear()

        #음식점, 카페, 숙박, 디저트, 엔터테인먼트,대중교통,공원,정원,주차시설,술집,화장실,관광명소,쇼핑 순으로 크롤링하는 함수
#         crawlNaverMap()
#         elem.send_keys(idx+' 주변 '+ search_kor[1])
        crawlNaverMap2()
        print('finish')
        n=add_dataframe(n,idx,r,c,dss,l,t,e,p,g,pl,b,tl,tr,s) #데이터프레임에 추가
        

읍내동 마애불상를 확인 중입니다.
읍내동 마애불상이/가 없습니다.
경복궁를 확인 중입니다.
경복궁이/가 존재합니다.
경복궁 주변 공원를 가져옵니다.
삼청공원
사직공원
수송공원
내수1근린공원
정동공원
청계천한빛광장
청진공원
인왕산공원
사직근린공원
월암근린공원
내수2근린공원
해맞이동산
무악공원
보신각공원
북악산공원
경복궁 주변 정원를 가져옵니다.
경복궁 주변 술집를 가져옵니다.
코블러
무드리
밥딜런앤더밴드
더구이
몽키비즈니스
비어수퍼
아주로
비어플러스
인산인해
인산인해
인산인해
인산인해
인산인해
인산인해
인산인해
인산인해
인산인해
비어플러스
인산인해
인산인해
인산인해
finish


In [34]:
print(idx+' 주변 '+ search_kor[0])

경복궁 주변 공원


In [37]:
print (r)

[]


In [38]:
print (c)

[]


In [15]:
print (l)

[]


In [16]:
print (dss)

[]


In [ ]:
print (e)

In [ ]:
print (t)

## 3.데이터프레임에 크롤링한 정보를 기록하기

In [ ]:
df.to_csv("Dongjak_Bigdata.csv",mode='w') #df를 csv파일로 저장하기

In [ ]:
df

In [115]:
elem=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-box/div/div/div/input')
elem.clear()
elem.send_keys(idx+' 주변 '+ search_kor[0])
# d.find_element_by_xpath('//*[@id="header"]/div[1]/fieldset/button').click()
d.find_element_by_xpath(NmapHome_searchbox).send_keys(Keys.ENTER) #엔터 치기
sleep(2)
i=0
#공원
print(idx+' 주변 '+search_kor[1]+'를 가져옵니다.') 
while True: #페이지 끝날때 까지
    try:
        for j in range(1,21):
            tag=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[2]/span').text
            if(tag=='근린공원'):
                garden=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/perfect-scrollbar/div/div[1]/div/div/div/search-item-place['+ str(j)+']/div/div[1]/div[1]/strong/span').text
                g.append(garden)
                print(garden)
                if j%3==0: #화면에 3개씩 보이도록 스크롤
                    ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
            else:
                if j%3==0: #화면에 3개씩 보이도록 스크롤
                    ELEMENT = d.find_elements_by_class_name('search_title_text')[j+1] 
                    d.execute_script("arguments[0].scrollIntoView(true);", ELEMENT)
        next_button=d.find_element_by_xpath('/html/body/app/layout/div[2]/div[2]/div[1]/shrinkable-layout/search-layout/search-list/search-list-contents/div/div[2]/button[2]')
        next_button.click()
        sleep(3)
    except: break
    

        


경복궁 주변 정원를 가져옵니다.
사직공원
수송공원
내수1근린공원
정동공원
청계천한빛광장
청진공원
인왕산공원
사직근린공원
월암근린공원
내수2근린공원
해맞이동산
무악공원
보신각공원
북악산공원
